In [85]:
import preprocessing
import pandas as pd
from data_reading.phyphox import get_experiments, read_experiment
listening_rate = 20

experiments = get_experiments()
data_frame = read_experiment(experiments[0])
data_frame = preprocessing.align_data(data_frame, listening_rate=listening_rate, reference_sensor=None)

video_length = 340
sample_rate = len(data_frame) / video_length
# TODO: load annotations
y = pd.read_csv("../../data/phyphox/long recording/annotations.tsv", delimiter="\t")
y = y.iloc[:, [3,5,8]]
y.columns = ["start", "end", "label"]
# iterate over the annotations and split the timeseries in chunks
chunks = [data_frame.iloc[int(annotation["start"]*sample_rate):int(annotation["end"]*sample_rate)] for i, annotation in y.iterrows()]
chunks = [c.reset_index(drop=True) for c in chunks]

/Users/julio/Documents/Uni/5_Master/Master Project/mp-mental-health-2020/src/preprocessing/_data_frame.py:31: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  time_delta_index = pd.TimedeltaIndex(timestamp_to_date, unit=output_timestamp_unit)


In [86]:
# TODO: replace label string values with indices
labels = y.loc[:, "label"].unique()
label_ids = { l: i for l,i in zip(labels, range(0,len(labels))) }

y = y.replace(label_ids)
for c,cl in zip(chunks, y.iterrows()):
    c["activity"] = int(cl[1]["label"])
    c["action_id"] = cl[0]

In [87]:
chunks_concat = pd.concat(chunks).reset_index(drop=True)
chunks_concat

,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,activity,action_id
0,0.035320,-0.013337,0.026752,-0.023912,-0.008865,0.006441,0,0
1,0.059538,0.018115,-0.006217,-0.016727,-0.003208,0.015903,0,0
2,-0.024354,0.020793,0.060347,-0.008347,-0.001332,0.018921,0,0
3,0.013580,0.013835,0.009312,-0.008660,0.001037,0.014761,0,0
4,0.084536,0.007026,0.070932,-0.009054,-0.004316,0.018061,0,0
...,...,...,...,...,...,...,...,...
291,0.973294,0.599652,-0.882202,0.129783,-0.905352,-0.760374,1,3
292,1.009741,0.447766,-0.913241,0.179277,-0.826804,-0.738206,1,3
293,0.973053,0.304662,-1.037655,0.234332,-0.764604,-0.700717,1,3
294,0.862463,0.108927,-1.131094,0.286549,-0.707382,-0.646120,1,3


In [88]:
%load_ext autoreload
%autoreload 2

labels = y.loc[:, "label"].squeeze()
print(labels)
from features._timeseries_feature_extraction import extract_timeseries_features
features = extract_timeseries_features(chunks_concat, labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0    0
1    0
2    1
3    1
Name: label, dtype: int64


Feature Extraction: 100%|██████████| 14/14 [00:01<00:00, 13.99it/s]


In [89]:
features.dropna(inplace=True, axis=1)
features

variable,acceleration_x__abs_energy,acceleration_x__absolute_sum_of_changes,"acceleration_x__agg_autocorrelation__f_agg_""mean""__maxlag_40","acceleration_x__agg_autocorrelation__f_agg_""median""__maxlag_40","acceleration_x__agg_autocorrelation__f_agg_""var""__maxlag_40","acceleration_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","acceleration_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","acceleration_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","acceleration_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","acceleration_x__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",...,gyroscope_z__symmetry_looking__r_0.9500000000000001,gyroscope_z__time_reversal_asymmetry_statistic__lag_1,gyroscope_z__time_reversal_asymmetry_statistic__lag_2,gyroscope_z__time_reversal_asymmetry_statistic__lag_3,gyroscope_z__value_count__value_-1,gyroscope_z__value_count__value_0,gyroscope_z__value_count__value_1,gyroscope_z__variance,gyroscope_z__variance_larger_than_standard_deviation,gyroscope_z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
0,7.573269,9.077713,-0.017216,0.017944,0.070973,0.307547,0.211388,0.049163,-0.004237,0.198426,...,1.0,-0.000002,-0.000020,-0.000076,0.0,0.0,0.0,0.007463,0.0,-8.093075
1,4.018051,9.473342,-0.022946,-0.005749,0.055162,0.054197,-0.017321,-0.112440,-0.019764,0.052775,...,1.0,-0.000402,-0.000680,-0.000907,0.0,0.0,0.0,0.006220,0.0,-1.654035
2,111.576428,20.666756,0.003634,-0.049475,0.128093,0.706487,-0.159126,-1.000492,0.349923,0.395465,...,1.0,-0.008482,-0.022055,-0.040174,0.0,0.0,0.0,0.299864,0.0,17.457897
3,436.098571,46.035608,-0.018003,-0.186820,0.192220,2.130552,0.609158,-1.748292,1.804764,1.735261,...,1.0,-0.004112,-0.015475,-0.034450,0.0,0.0,0.0,1.051438,1.0,4.399761


In [91]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [('Logistic Regression', LogisticRegression(solver='liblinear', multi_class='ovr')), ('LDA', LinearDiscriminantAnalysis()), ('LinearSVC', LinearSVC()), ('CART', DecisionTreeClassifier()), ('NB', GaussianNB())]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
for name, model in models:
    print(name)
    scores = cross_val_score(model, X_scaled, labels, cv=2)
    print(scores)

Logistic Regression
[1. 1.]
LDA
[nan nan]
LinearSVC
[1. 1.]
CART
[0.  0.5]
NB
[0.5 1. ]


/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of samples must be more than the number of classes.

  FitFailedWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The number of samples must be more than the number of classes.

  FitFailedWarning)
